In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as plt

In [ ]:
df_1 = pd.read_csv(r"..\..\data\clean\all_clients.csv")
df_2 = pd.read_csv(r"..\..\data\clean\control_filter.csv")
df_3 = pd.read_csv(r"..\..\data\clean\test_filter.csv")

In [ ]:
"""
input: dataframe
output: dataframe

function to only take latest session per client. previous sessions will be removed

"""

def valid_session(df: pd.DataFrame) -> pd.DataFrame:
    # Convert the 'date_time' column to datetime format
    df['date_time'] = pd.to_datetime(df['date_time'])
    
    # Get the most recent session for each client (the latest visit_id per client)
    most_recent_sessions = df.loc[df.groupby('client_id')['date_time'].idxmax()]
    
    # Now filter the data to include only the most recent sessions
    df_recent = df[df['visit_id'].isin(most_recent_sessions['visit_id'])]
    
    # Sort by client_id, visitor_id, and date_time to make sure steps are in correct order
    df_recent = df_recent.sort_values(by=['client_id', 'visitor_id', 'date_time'])

    return df_recent

In [ ]:
df_1.shape

In [ ]:
df_1 = valid_session(df_1)
df_1.shape

In [ ]:
##--- this is outdated ---

In [ ]:
#concat both footprint csvs for a whole set and dropping duplicate rows
df_df = pd.concat([df_df_1_org, df_df_2_org], axis=0, ignore_index=True)
df_df = df_df.drop_duplicates()

#sorting the date to assure the valid process_step is always the first to shown; valid is defined as "most recent"
df_df.head()
df_df_sorted_date = df_df.sort_values(by=["client_id", "visitor_id", 'date_time'], ascending=True)

print("\n", df_df.shape)
df_df.head(20)

#quick check if all entries are within the duration of experiment
min(df_df["date_time"])
max(df_df["date_time"])



In [ ]:
"""
input: dataframe
output: dataframe

function to ONLY take the latest date_time row for EACH process_step PER client_id

"""

def validating_steps(df):
    df['date_time'] = pd.to_datetime(df['date_time'])
    process_steps = ['confirm', 'step_3', 'step_2', 'step_1', 'start']

    df_filtered = df[df['process_step'].isin(process_steps)]

    latest_rows = df_filtered.loc[df_filtered.groupby(['client_id', 'process_step'])['date_time'].idxmax()]
    latest_rows = latest_rows.sort_values(by="date_time", ascending = True)
    
    return latest_rows

#counting the number of valid process_steps to see if the filtering might had issues: 
#start > step_1 > step_2 > step_3 > confirm
valid_df_total = validating_steps(df_df)
valid_df_total["process_step"].value_counts(), valid_df_total.shape


In [ ]:
df_cp = df_cp_org.copy()
df_cp.head()

In [ ]:
#applying the filter of valid rows to client profile dataframe

df_cp_valid = pd.merge(valid_df_total, df_cp, how="left", on="client_id")
df_cp_valid

In [ ]:
# importing experiment roster and removing invalid rows (no group assigned)

df_er = df_er_org.copy()
df_er = df_er.dropna()
df_er.head(), df_er.shape

In [ ]:
#seperating the total valid df into testing and control group
testing = df_er[df_er["Variation"] == "Test"]
testing = pd.merge(testing, df_cp_valid, how="left", on="client_id")
testing.columns = testing.columns.str.lower().str.lstrip()

control = df_er[df_er["Variation"] == "Control"]
control = pd.merge(control, df_cp_valid, how="left", on="client_id")
control.columns = control.columns.str.lower().str.lstrip()



control.variation.unique()
control.shape
control.head(30)

In [ ]:
testing = pd.merge(df_er, df_cp_valid, how="left", on="client_id")
testing